In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

#trainset = datasets.MNIST('/kaggle/input/digit-recognizer/train.csv', download = True, train = True, transform = transform)
#trainset = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
#trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
print("downloaded")

X = train.loc[:,train.columns != "label"].values/255
Y = train.label.values
#features_test = test.loc[:,test.columns != "label"].values/255
#targets_test = test.label.values

print("0")
features_train, features_test, targets_train, targets_test = train_test_split(X, Y, test_size=0.25, random_state = 42)
print("1")
X_train = torch.from_numpy(features_train)
X_test = torch.from_numpy(features_test)

Y_train = torch.from_numpy(targets_train)
Y_test = torch.from_numpy(targets_test)

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_test, Y_test)
print("2")
trainloader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(test, batch_size = 64, shuffle = True)

In [ ]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
print(type(images))
images = images.view(images.shape[0], -1)

from torch import optim
optimizer = optim.SGD(model.parameters(),lr=0.01)
#optimizer.zero_grad()
model = model.float()
epochs = 15
for e in range(epochs):
    print("E",e)
    training_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model.forward(images.float())
        
        loss = criterion(output, labels)
        loss.backward()
        
        optimizer.step()
        training_loss += loss.item()
    else:
        accuracy = 0
        test_loss = 0
        print("Training_loss", training_loss/len(trainloader))
        with torch.no_grad():
            for images, labels in testloader:
                images = images.view(images.shape[0], -1)
                labels = Variable(labels)
                output = model.forward(images.float())
                
                test_loss += criterion(output, labels)
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim =1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
   # model.train()
    print("Test Loss =", test_loss/len(testloader))
    print("Accuracy", accuracy)
        

In [ ]:
test_images = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

test_image = test_images.loc[:,test_images.columns != "label"].values/255
test_dataset = torch.from_numpy(test_image)

new_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [ ]:
results = []
with torch.no_grad():
    for images in new_test_loader:
        test = images.view(images.shape[0], -1)
        output = model(test.float())
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim = 1)
        results += top_class.numpy().tolist()

In [ ]:
predictions = np.array(results).flatten()

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("my_submissions.csv", index=False, header=True)